<a href="https://colab.research.google.com/github/rahulkher/data-science-ideas/blob/main/Criminal_Proviosions_mistralv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers>=4.41.0 accelerate==0.26.1 bitsandbytes huggingface_hub>=0.24.0

In [2]:
import math
import torch

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
import kagglehub
from huggingface_hub import login
from google.colab import userdata

import pandas as pd
import numpy as np

import re
import orjson as _jsonlib
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os

folder_path = '/content/drive/MyDrive/Central acts'

pdf_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.pdf')]
print(len(pdf_files))

df = pd.read_csv('/content/drive/MyDrive/Central acts/acts_dataset.csv')
print(df.shape)

Mounted at /content/drive
102
(1432, 7)


In [4]:
login(token = userdata.get("hf_token"))
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype='float16',
    bnb_4bit_use_double_quant=False
)

mistral7b_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    max_length=6,use_fast= False,
    padding_side="left"
)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [6]:
SYS_PROMPT_TEMPLATE = """

You are a legal analysis assistant.
I will give you the text of a legal provision.
Your task is to analyze it and return a structured JSON object determining whether it is a criminal provision or not.

Analysis Steps:
	1.	Offence-Creating Language — Check if it uses words like “Whoever”, “commits an offence”, “shall be punished”, “is guilty of”, or “liable to imprisonment/fine”.
	2.	Criminal Punishments — Identify if it mentions imprisonment, criminal fines, or both.
	3.	Criminal Procedure References — Look for references to CrPC, FIR, cognizable/non-cognizable, bailable/non-bailable, trial.
	4.	Statutory Context — Determine if it’s part of a penal code or the “Offences and Penalties” section of a special law.
	5.	Prosecutor — Check if the prosecution would be done by the State in criminal court.

Classification Rules:
	•	If imprisonment or criminal procedure is involved → “Criminal”.
	•	If only civil penalties, compensation, injunctions, or administrative fines → “Not Criminal”.
	•	If uncertain → “Likely Criminal” or “Likely Not Criminal” (with explanation).

Output Format (JSON only, no extra text):

{
  "classification": "Criminal / Not Criminal / Likely Criminal / Likely Not Criminal",
  "reasoning": "Brief reasoning here",
  "indicators": {
    "offence_creating_language": true/false,
    "criminal_punishments": true/false,
    "criminal_procedure_references": true/false,
    "statutory_context": "string describing law context",
    "state_prosecution_likely": true/false
  }
}

Always return the complete JSON and do not add any additional parenthisis or brackets other than what i have specified above
"""

In [7]:
def generate_completion(user_text: str) -> str:
    messages = [
        {"role":"system","content": SYS_PROMPT_TEMPLATE},
        {"role":"user","content": user_text}
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(mistral7b_model.device)
    with torch.no_grad():
        out_tokens = mistral7b_model.generate(
            **inputs,
            max_new_tokens=1024,
            # temperature=0.0,  # deterministic for classification
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(out_tokens[0], skip_special_tokens=True)

In [8]:
def to_json_obj(s: str):
      # 1) Drop all [/INST]
      s = s.replace('[/INST]', '')

      # 2) Keep only the outermost JSON block
      start = s.find('{')
      end = s.rfind('}')
      if start == -1 or end == -1 or end < start:
          raise ValueError("No JSON object found")
      chunk = s[start:end+1]

      # 3) Remove any non-JSON junk between a closing '}' and the final '}' (your sentence)
      #    e.g.  ... } . remove ... JSON \n }
      chunk = re.sub(r'\}\s*[^{}]*\s*\}', r'}}', chunk)

      # 4) Optional: remove trailing commas before } or ]
      chunk = re.sub(r',\s*([}\]])', r'\1', chunk)

      # 5) Parse
      return _jsonlib.loads(chunk)

In [9]:
# Compile regex once outside the loop for performance
_json_pat = re.compile(r"\[/INST\]\s*\[?\{.*\}\]?", re.DOTALL)

# Create empty columns
df['classification'] = None
df['reasoning'] = None
df['offence_creating_language'] = None
df['criminal_punishments'] = None
df['criminal_procedure_references'] = None
df['statutory_context'] = None
df['state_prosecution_likely'] = None

for idx, i in tqdm(enumerate(df.text), total=len(df), desc="Processing provisions"):
    user_text = f"Please classify the provision\n{i}\n"
    outcome = generate_completion(user_text)

    m = _json_pat.search(outcome)
    if m:
        try:
            obj = to_json_obj(m.group(0))
            df.at[idx, 'classification'] = obj.get('classification')
            df.at[idx, 'reasoning'] = obj.get('reasoning')
            df.at[idx, 'offence_creating_language'] = obj['indicators'].get('offence_creating_language')
            df.at[idx, 'criminal_punishments'] = obj['indicators'].get('criminal_punishments')
            df.at[idx, 'criminal_procedure_references'] = obj['indicators'].get('criminal_procedure_references')
            df.at[idx, 'statutory_context'] = obj['indicators'].get('statutory_context')
            df.at[idx, 'state_prosecution_likely'] = obj['indicators'].get('state_prosecution_likely')
        except Exception as e:
            df.at[idx, 'classification'] = outcome
    else:
        df.at[idx, 'classification'] = outcome

df.to_csv('/content/drive/MyDrive/Central acts/acts_dataset_classified.csv', index=False)

Processing provisions: 100%|██████████| 1432/1432 [5:52:37<00:00, 14.78s/it]


In [22]:
_json_pat = re.compile(r"\[/INST\]\s\{.*\}", re.DOTALL)
m = _json_pat.search(generate_completion(user_text))
def to_json_obj(s: str):
    # 1) Drop all [/INST]
    s = s.replace('[/INST]', '')

    # 2) Keep only the outermost JSON block
    start = s.find('{')
    end = s.rfind('}')
    if start == -1 or end == -1 or end < start:
        raise ValueError("No JSON object found")
    chunk = s[start:end+1]

    # 3) Remove any non-JSON junk between a closing '}' and the final '}' (your sentence)
    #    e.g.  ... } . remove ... JSON \n }
    chunk = re.sub(r'\}\s*[^{}]*\s*\}', r'}}', chunk)

    # 4) Optional: remove trailing commas before } or ]
    chunk = re.sub(r',\s*([}\]])', r'\1', chunk)

    # 5) Parse
    return _jsonlib.loads(chunk)
obj = to_json_obj(m.group(0))

print(obj)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'classification': 'Criminal', 'reasoning': "The provision uses offence-creating language ('Every person shall be liable to punishment under this Sanhita and not otherwise for every act or omission contrary to the provisions thereof, of which he shall be guilty within India.'), criminal procedure references (mentions trial and conviction), and statutory context (part of a penal code - Bharatiya Nyaya Sanhita, 2023).", 'indicators': {'offence_creating_language': True, 'criminal_punishments': True, 'criminal_procedure_references': True, 'statutory_context': 'Bharatiya Nyaya Sanhita, 2023', 'state_prosecution_likely': True}}


In [12]:
df[df['classification']=='Criminal']

,Unnamed: 0,file_path,relative_path,file_name,page,text,ocr_used,classification,reasoning,offence_creating_language,criminal_punishments,criminal_procedure_references,statutory_context,state_prosecution_likely
